# Pusula Data Science Intern Case (2025)
Bu notebook, verilen **Excel veri seti** üzerinde EDA (keşifsel veri analizi) ve veri ön işleme (preprocessing) adımlarını yapar. `TedaviSuresi` ve `UygulamaSuresi` sütunları içindeki sayılar ayıklanarak doğru analiz yapılması sağlanmıştır. Çalıştırma ortamı: Google Colab.

In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn openpyxl joblib

## 1. Veri Yükleme

In [ ]:
import pandas as pd

df = pd.read_excel("Talent_Academy_Case_DT_2025.xlsx")

print(df.shape)
df.head()

## 2. Keşifsel Veri Analizi (EDA) – Sayı Ayıklama ile

In [ ]:
import re
import matplotlib.pyplot as plt
import seaborn as sns

# Sayı ayıklama fonksiyonu
def extract_number(x):
    if pd.isna(x):
        return None
    match = re.search(r"\d+", str(x))
    if match:
        return int(match.group())
    return None

# TedaviSuresi ve UygulamaSuresi'ni sayıya çevir
df["TedaviSuresi"] = df["TedaviSuresi"].apply(extract_number)
df["UygulamaSuresi"] = df["UygulamaSuresi"].apply(extract_number)

# Eksik değer oranları
print(df.isna().mean().sort_values(ascending=False).head(20))

# Sayısal kolonlar
num_cols = ["Yas","TedaviSuresi","UygulamaSuresi"]
df[num_cols].hist(bins=30, figsize=(12,6))
plt.show()

# Korelasyon
sns.heatmap(df[num_cols].corr(), annot=True, cmap="Blues")
plt.show()

# Kategorik dağılımlar
for col in ["Cinsiyet","KanGrubu","Uyruk","Bolum","TedaviAdi"]:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        df[col].value_counts().head(15).plot(kind="bar")
        plt.title(col)
        plt.show()

## 3. Veri Ön İşleme (Preprocessing)

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

def normalize_whitespace(x):
    if pd.isna(x): return x
    return re.sub(r"\s+", " ", str(x).strip())

def split_multivalue(cell):
    if pd.isna(cell): return []
    parts = re.split(r"[;,]", str(cell))
    return [normalize_whitespace(p) for p in parts if p.strip()]

class MultiLabelBinarizerDF(BaseEstimator, TransformerMixin):
    def __init__(self, columns): self.columns = columns
    def fit(self, X, y=None):
        self.classes_ = {}
        for col in self.columns:
            items = set()
            for cell in X[col].fillna(""):
                items.update(split_multivalue(cell))
            self.classes_[col] = sorted(items)
        return self
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            for val in self.classes_[col]:
                X[f"{col}__{val}"] = X[col].map(lambda x: 1 if val in split_multivalue(x) else 0)
            X.drop(columns=[col], inplace=True)
        return X

target = "TedaviSuresi"
X = df.drop(columns=[target])
y = df[target]

single_cat = ["Cinsiyet","KanGrubu","Uyruk","Bolum","TedaviAdi"]
multi_cat = ["KronikHastalik","Alerji","Tanilar","UygulamaYerleri"]
numeric = ["Yas","UygulamaSuresi"]

numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
cat_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", OneHotEncoder(handle_unknown="ignore"))])

mlb = MultiLabelBinarizerDF(columns=multi_cat)
X_multi = mlb.fit_transform(X)

coltrans = ColumnTransformer([
    ("num", numeric_tf, numeric + [c for c in X_multi.columns if "__" in c]),
    ("cat", cat_tf, single_cat)
])

pipe = Pipeline([("multilabel", mlb), ("coltrans", coltrans)])

X_ready = pipe.fit_transform(X)
print("Model-ready shape:", X_ready.shape)

## 4. Çıktıları Kaydetme

In [ ]:
import joblib

pd.DataFrame(X_ready).to_csv("dataset_model_ready.csv", index=False)
joblib.dump(pipe, "model_ready_preprocess.joblib")

print("Kaydedildi")